# Standard Name Table for PIV

In [1]:
from ssnolib import parse_table, StandardNameTable, AgentRole, StandardName, VectorStandardName
from ontolutils.ex.prov import Organization, Person
from datetime import datetime
import ontolutils

In [2]:
zenodo_id = 17550472

In [3]:
VERSION = "1.2.0"

In [4]:
description = """This table provides standard names for the description of PIV data. It is an initial version covering the most basic properties of planar PIV data."""

In [5]:
changelog = """
v1.1.0: Improved some standard name descriptions
v1.2.0: uses schema:version instead of dcterms:hasversion
"""

In [6]:
snt = StandardNameTable(
    title="Standard Name Table for Particle Image Velociemtry data",
    version=VERSION,
    description=description,
    id=f"https://doi.org/10.5281/zenodo.{zenodo_id}",
    # editorial_note="Changed blank nodes to explicit IRIs. This especially makes the standard names unique for the version and allows better visualization in tools like GraphDB. The IRIs uses the Zenodo Record URL as a prefix.",
    created="2024-11-17",
    modified=datetime.today().strftime("%Y-%m-%d")
)
print(snt.serialize("ttl", base_uri="https://doi.org/10.5281/zenodo.{zenodo_id}"))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix schema: <https://schema.org/> .
@prefix ssno: <https://matthiasprobst.github.io/ssno#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://doi.org/10.5281/zenodo.17550472> a ssno:StandardNameTable ;
    dcterms:created "2024-11-17"^^xsd:date ;
    dcterms:description "This table provides standard names for the description of PIV data. It is an initial version covering the most basic properties of planar PIV data." ;
    dcterms:modified "2025-11-07"^^xsd:date ;
    dcterms:title "Standard Name Table for Particle Image Velociemtry data" ;
    schema:version "1.2.0" .




In [7]:
# with ontolutils.set_config(blank_node_prefix_name="local:"):
KIT = Organization(id="https://ror.org/04t3en479",
                   name="Karlsruhe Institute of Technology, Institute of Thermal Turbomachinery",
                   ror_id="https://ror.org/04t3en479")
author1 = Person(id="https://orcid.org/0000-0001-8729-0482",
                 first_name="Matthias",
                 last_name="Probst",
                 orcid_id="https://orcid.org/0000-0001-8729-0482",
                affiliation=KIT)
author2 = Person(id="https://orcid.org/0000-0001-9560-500X",
                 first_name="Balazs",
                 last_name="Pritz",
                 orcid_id="https://orcid.org/0000-0001-9560-500X",
                 affiliation=KIT)

In [8]:
# with ontolutils.set_config(blank_node_prefix_name="local:"):
snt.add_author(author1, AgentRole.Contact_Person)
snt.add_author(author2, AgentRole.Supervisor)

## Adding Standard names

In [9]:
import pathlib
from typing import Union

import yaml


def read_standard_names_from_yaml(yaml_filename: Union[str, pathlib.Path]):
    yaml_filename = pathlib.Path(yaml_filename)
    if not yaml_filename.exists():
        raise FileNotFoundError(f"File {yaml_filename} does not exist")

    with open(yaml_filename, 'r') as f:
        data = yaml.safe_load(f)

    standard_names = []
    for k, v in data.items():
        standard_names.append(StandardName(standard_name=k, unit=v["unit"], description=v["description"]))

    return standard_names

In [10]:
# with ontolutils.set_config(blank_node_prefix_name="local:"):
standard_names = read_standard_names_from_yaml("piv_standard_names.yaml")

In [11]:
def improve_id(standard_name):
    standard_name.id = f"{snt.id}#{standard_name.standardName}"
    return standard_name
    

In [12]:
snt.standardNames = [improve_id(sn) for sn in standard_names]

In [13]:
file_name = "Standard_Name_Table_for_Particle_Image_Velocimetry"

In [14]:
json_ld_filename = snt.to_jsonld(
    f"{file_name}.jsonld",
    base_uri=snt.id + "#",
    context={"local": f"https://doi.org/10.5281/zenodo.{zenodo_id}#"},
    overwrite=True)
ttl_filename = snt.to_ttl(
    json_ld_filename.with_suffix(".ttl"),
    base_uri=snt.id + "#",
    context={"local": f"https://doi.org/10.5281/zenodo.{zenodo_id}#"},
    overwrite=True
)
xml_filename = snt.to_xml(
    json_ld_filename.with_suffix(".xml"),
    base_uri=snt.id + "#",
    context={"local": f"https://doi.org/10.5281/zenodo.{zenodo_id}#"},
    overwrite=True
)